In [1]:
import quilt3 as q3

## `browse`

In [6]:
p = q3.Package.browse("aics/label-free-imaging-collection", registry="s3://allencell")
p

Loading manifest: 100%|██████████████████████| 49342/49342 [00:00<00:00, 111k/s]


(remote Package)
 └─README.md
 └─cells_2d/
   └─fov-0_CellIndex-11.tiff
   └─fov-0_CellIndex-12.tiff
   └─fov-0_CellIndex-5.tiff
   └─fov-0_CellIndex-6.tiff
   └─fov-0_CellIndex-7.tiff
   └─fov-0_CellIndex-9.tiff
   └─fov-1000_CellIndex-1.tiff
   └─fov-1000_CellIndex-5.tiff
   └─fov-1001_CellIndex-2.tiff
   └─fov-1001_CellIndex-5.tiff
   └─fov-1001_CellIndex-8.tiff
   └─fov-1001_CellIndex-9.tiff
   └─fov-1002_CellIndex-2.tiff
   └─fov-1003_CellIndex-1.tiff
 └─cells_3d/
 └─data_feats.csv
 └─fovs/
 └─reference_images/
 ...

## pull S3 files into pandas as a df

In [7]:
p["data_feats.csv"]()

,Unnamed: 0,CellId,actin_dna_cell_struct_cell_distal_ratio,actin_dna_cell_struct_cell_maj_ratio,actin_dna_cell_struct_cell_min_ratio,actin_dna_cell_struct_cell_z_ratio,actin_dna_cell_struct_cyto_distal_ratio,actin_dna_cell_struct_cyto_maj_ratio,actin_dna_cell_struct_cyto_min_ratio,actin_dna_cell_struct_cyto_z_ratio,...,tl_dna_cell_struct_cyto_distal_ratio,tl_dna_cell_struct_cyto_maj_ratio,tl_dna_cell_struct_cyto_min_ratio,tl_dna_cell_struct_cyto_z_ratio,tl_dna_cell_struct_nuc_distal_ratio,tl_dna_cell_struct_nuc_maj_ratio,tl_dna_cell_struct_nuc_min_ratio,tl_dna_cell_struct_nuc_z_ratio,CellIndex,source_image
0,0,3,0.955416,0.269468,0.359371,0.181782,0.543976,0.276367,0.345835,0.204207,...,0.216265,0.059198,0.007891,0.385714,-0.271866,0.029331,0.078083,-0.271512,3,predicted_6522.tiff
1,1,4,0.886773,0.004524,0.163753,-0.031596,0.273288,0.025962,0.195265,0.013440,...,0.225798,0.168526,0.384287,0.260535,-0.343634,0.049786,0.107829,-0.227123,4,predicted_6522.tiff
2,2,8,0.911882,0.063944,0.166627,-0.046949,0.406951,0.021671,0.144362,-0.042678,...,0.180613,0.180101,0.207042,0.271698,-0.361285,0.000713,0.002995,-0.258752,8,predicted_6522.tiff
3,3,9,0.821321,0.098141,0.147941,0.033536,0.081553,0.029719,0.121659,0.074720,...,0.116383,0.171442,0.259956,0.371761,-0.310801,0.007754,0.064016,-0.165579,9,predicted_6522.tiff
4,4,20,0.921988,0.302046,0.646655,-0.141504,0.491229,0.321640,0.648595,-0.109941,...,0.078222,0.248311,0.369813,0.456255,-0.354896,0.091899,0.062955,-0.258185,3,predicted_6523.tiff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20565,20565,84303,0.991595,0.127135,0.612907,-0.051105,0.173965,0.128741,0.616112,-0.051223,...,0.109489,0.004942,0.559133,0.385160,-0.262266,0.020377,0.053209,-0.203199,6,predicted_1619.tiff
20566,20566,84304,0.949586,0.545224,0.395418,0.286573,0.279358,0.533825,0.411620,0.296553,...,0.095950,0.059704,0.337312,0.474595,-0.276509,0.027343,0.015527,-0.084296,7,predicted_1619.tiff
20567,20567,84306,0.874008,0.343473,0.571956,-0.184323,0.401922,0.312948,0.588863,-0.169249,...,0.116053,0.239909,0.236472,0.375847,-0.288288,0.105293,0.091827,-0.181218,2,predicted_1620.tiff
20568,20568,84310,0.888927,0.348529,0.223149,0.121953,0.402357,0.311608,0.244212,0.171863,...,0.171095,0.310661,0.227441,0.409762,-0.290950,0.014814,0.086252,-0.216949,6,predicted_1620.tiff


## `filter` packages

You could also filter on `e.meta` (object-level metadata) if present

In [4]:
pf = p.filter(lambda l, e: l.startswith("cells_2d"))
pf

(remote Package)
 └─cells_2d/
   └─fov-0_CellIndex-11.tiff
   └─fov-0_CellIndex-12.tiff
   └─fov-0_CellIndex-5.tiff
   └─fov-0_CellIndex-6.tiff
   └─fov-0_CellIndex-7.tiff
   └─fov-0_CellIndex-9.tiff
   └─fov-1000_CellIndex-1.tiff
   └─fov-1000_CellIndex-5.tiff
   └─fov-1001_CellIndex-2.tiff
   └─fov-1001_CellIndex-5.tiff
   └─fov-1001_CellIndex-8.tiff
   └─fov-1001_CellIndex-9.tiff
   └─fov-1002_CellIndex-2.tiff
   └─fov-1003_CellIndex-1.tiff
   └─fov-1003_CellIndex-12.tiff
   └─fov-1003_CellIndex-13.tiff
   └─fov-1003_CellIndex-15.tiff
   └─fov-1003_CellIndex-2.tiff
   └─fov-1003_CellIndex-5.tiff
 ...

## set object-level metadata

In [14]:
pf["cells_2d"]["fov-0_CellIndex-11.tiff"].set_meta({"tags": ["blue", "yellow"], "bounding_box": [[225,235], [306,345]]})

In [15]:
pf["cells_2d"]["fov-0_CellIndex-11.tiff"].meta

{'tags': ['blue', 'yellow'], 'bounding_box': [[225, 235], [306, 345]]}

## package-level metadata also exists

In [19]:
pf.set_meta({"description": "imaging collection"})
pf.meta

{'description': 'imaging collection'}

## `push` the result back to S3
* Now available for SQL querying via Athena
   * There is a view of all packaged objects, a view of all packages

You can then do things like this to work with arbitrary data on the fly.
```sql
SELECT * FROM "YOUR-BUCKET_objects-view"
WHERE substr(logical_key, -5) = '.tiff'
-- extract and query object-level metadata
AND json_extract_scalar(meta, '$.user_meta.nucmembsegmentationalgorithmversion') LIKE '1.3%'
AND json_array_contains(json_extract(meta, '$.user_meta.cellindex'), '5');
```

See also [Quilt Athena docs](https://docs.quiltdata.com/advanced/athena#example-query-object-level-metadata)

In [20]:
# e.g. pf.push("pdb-experiment/2024", ...)